# Multi Model Evaluation

In [1]:
import sys
sys.path.append("..")

import copy
import cProfile
from datasets import load_dataset
import itertools
import json
import math
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
import random
from sklearn.metrics import classification_report, accuracy_score, f1_score
import time
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, DataCollatorWithPadding, \
                         AutoModelForSequenceClassification, BertForSequenceClassification, \
                         BertModel, RobertaForSequenceClassification, RobertaModel

from resilient_nlp.mini_roben import Clustering, ClusterRepRecoverer, ClusterRecovererWithPassthrough
from resilient_nlp.models import BertClassifier
from resilient_nlp.perturbers import ToyPerturber, WordScramblerPerturber
from runner import ExperimentRunner
from word_score_attack import BertWordScoreAttack

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
#tasks = ('imdb', 'sst', 'sst_bin', 'yelp_bin', 'yelp_full')
#tasks = ('sst_bin', 'yelp_bin', 'yelp_full')
tasks = ('yelp_bin',)

In [3]:
model_types = ('bert', 'roberta')

Config for final evaluation on test set

In [4]:
eval_set_size = 500
use_dev_set = False

Config for evaluation on dev set

In [5]:
#eval_set_size = 113
#use_dev_set = True

In [6]:
max_raw_length = 826
preprocess = lambda row: { 'text': row['text'].lower()[:max_raw_length]}

## IMDb Dataset

In [7]:
sampled_test_set = {}
sampled_test_set_dict = {}
sampled_test_set_adv_no_ws = {}
sampled_test_set_adv_incl_ws = {}

In [8]:
imdb = load_dataset('artemis13fowl/imdb')

Using custom data configuration artemis13fowl--imdb-f63738dec0d5e230
Reusing dataset parquet (/home/jasko/.cache/huggingface/datasets/parquet/artemis13fowl--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
random.seed(11)
if use_dev_set:
    sampled_test_set['imdb'] = imdb['dev'].select(random.choices(range(len(imdb['dev'])), k=eval_set_size)).map(preprocess)
else:
    sampled_test_set['imdb'] = imdb['attack_eval_truncated'].select(range(eval_set_size)).map(preprocess)


# This is silly but apparently huggingface datasets are immutable?
# Representing it as something a bit more sane
sampled_test_set_dict['imdb'] = [
    {
        'text': row['text'],
        'label': row['label'],
    }
    for row in sampled_test_set['imdb']
]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/parquet/artemis13fowl--imdb-f63738dec0d5e230/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901/cache-b45d493b37ab1dc7.arrow


## SST-5 Dataset

In [10]:
treebank_detok = TreebankWordDetokenizer()

sst = load_dataset('sst').map(
    lambda row: {
        "text": treebank_detok.detokenize(row["sentence"].split()),
        "label": min(math.floor(row["label"] / 0.2), 4.0),
    }, remove_columns=['sentence', 'tokens', 'tree']
)

random.seed(11)
if use_dev_set:
    sampled_test_set['sst'] = sst['validation'].select(random.choices(range(len(sst['validation'])), k=eval_set_size)).map(preprocess)
else:
    sampled_test_set['sst'] = sst['test'].select(random.choices(range(len(sst['test'])), k=eval_set_size)).map(preprocess)

sampled_test_set_dict['sst'] = [
    {
        'text': row['text'],
        'label': row['label'],
    }
    for row in sampled_test_set['sst']
]

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-3c142acdab53f98c.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-0bf56ce0086915ee.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-19fdf8d124be4ba7.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-fc3e75429c3f7637.arrow


## SST-2 Dataset

In [11]:
treebank_detok = TreebankWordDetokenizer()

sst_bin = load_dataset('sst').filter(
        lambda row: row["label"] < 0.4 or row["label"] >= 0.6
    ).map(
    lambda row: {
        "text": treebank_detok.detokenize(row["sentence"].split()),
        "label": min(math.floor(row["label"] / 0.5), 1.0),
    }
)

random.seed(11)
if use_dev_set:
    sampled_test_set['sst_bin'] = sst_bin['validation'].select(random.choices(range(len(sst_bin['validation'])), k=eval_set_size)).map(preprocess)
else:
    sampled_test_set['sst_bin'] = sst_bin['test'].select(random.choices(range(len(sst_bin['test'])), k=eval_set_size)).map(preprocess)

sampled_test_set_dict['sst_bin'] = [
    {
        'text': row['text'],
        'label': row['label'],
    }
    for row in sampled_test_set['sst_bin']
]

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-f4f1ada73617d193.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6279b6f0f8a08f9a.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6c5f77e5aefdd0e2.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-c46f07c913633b4d.arrow
Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-87779081ead23eae.arrow
Loading cached processed dataset at /home/jasko/.cache/huggi

## Yelp-2

In [12]:
yelp_bin = load_dataset('yelp_polarity')

random.seed(11)
sampled_test_set['yelp_bin'] = yelp_bin['test'].select(random.choices(range(len(yelp_bin['test'])), k=eval_set_size)).map(preprocess)

sampled_test_set_dict['yelp_bin'] = [
    {
        'text': row['text'],
        'label': row['label'],
    }
    for row in sampled_test_set['yelp_bin']
]

Reusing dataset yelp_polarity (/home/jasko/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544/cache-4aafc3a0650c7800.arrow


## Yelp-5

In [13]:
yelp_full = load_dataset('yelp_review_full')

random.seed(11)
sampled_test_set['yelp_full'] = yelp_full['test'].select(random.choices(range(len(yelp_full['test'])), k=eval_set_size)).map(preprocess)

sampled_test_set_dict['yelp_full'] = [
    {
        'text': row['text'],
        'label': row['label'],
    }
    for row in sampled_test_set['yelp_full']
]

Reusing dataset yelp_review_full (/home/jasko/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/13c31a618ba62568ec8572a222a283dfc29a6517776a3ac5945fb508877dde43)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jasko/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/13c31a618ba62568ec8572a222a283dfc29a6517776a3ac5945fb508877dde43/cache-be339907fd6abb29.arrow


### Perturbations

In [14]:
def generate_perturbed_multiset(input, wsp):
    random.seed(11)
    result = []

    for i in range(10):
        test_item = copy.deepcopy(input)

        for row in test_item:
            row['text'] = wsp.perturb([row['text']])[0][0]
        result.append(test_item)
    
    return result

Perturbed set with no whitespace modifications

In [15]:
wsp = WordScramblerPerturber(perturb_prob=0.1, weight_add=1, weight_drop=1, weight_swap=1,
                             weight_split_word=0, weight_merge_words=0)

for task in tasks:
    sampled_test_set_adv_no_ws[task] = generate_perturbed_multiset(sampled_test_set_dict[task], wsp)

Perturbed set with whitespace modifications

In [16]:
wsp = WordScramblerPerturber(perturb_prob=0.1, weight_add=1, weight_drop=1, weight_swap=1,
                             weight_split_word=1, weight_merge_words=1)

for task in tasks:
    sampled_test_set_adv_incl_ws[task] = generate_perturbed_multiset(sampled_test_set_dict[task], wsp)

## Models

### BERT, including finetuned variants

In [17]:
tokenizer = {}
model_base = {}
model_finetuned = { type: {} for type in model_types }
model_finetuned_all_pert = { type: {} for type in model_types }

In [18]:
bert_checkpoint = "bert-base-uncased"
tokenizer['bert'] = AutoTokenizer.from_pretrained(bert_checkpoint)
model_base['bert'] = BertModel.from_pretrained(bert_checkpoint).to(device)

In [19]:
bert_checkpoint_finetuned_imdb = "artemis13fowl/bert-base-uncased-imdb"
if 'bert' in model_types and 'imdb' in tasks:
    model_finetuned['bert']['imdb'] = BertForSequenceClassification.from_pretrained(bert_checkpoint_finetuned_imdb).to(device)

In [20]:
bert_checkpoint_finetuned_imdb_all_pert = "jjezabek/bert-base-uncased-imdb-all-pert"
if 'bert' in model_types and 'imdb' in tasks:
    model_finetuned_all_pert['bert']['imdb'] = BertForSequenceClassification.from_pretrained(bert_checkpoint_finetuned_imdb_all_pert).to(device)

In [21]:
bert_checkpoint_finetuned_sst = "jjezabek/bert-base-uncased-sst"
if 'bert' in model_types and 'sst' in tasks:
    model_finetuned['bert']['sst'] = BertForSequenceClassification.from_pretrained(bert_checkpoint_finetuned_sst).to(device)

In [22]:
bert_checkpoint_finetuned_sst_bin = '/home/jasko/resilient_nlp/output/bert-base-uncased-sst_bin/checkpoint-800'
if 'bert' in model_types and 'sst_bin' in tasks:
    model_finetuned['bert']['sst_bin'] = BertForSequenceClassification.from_pretrained(bert_checkpoint_finetuned_sst_bin).to(device)

In [23]:
bert_checkpoint_finetuned_yelp_bin = '/home/jasko/resilient_nlp/output/bert-base-uncased-yelp_bin/checkpoint-3500'
if 'bert' in model_types and 'yelp_bin' in tasks:
    model_finetuned['bert']['yelp_bin'] = BertForSequenceClassification.from_pretrained(bert_checkpoint_finetuned_yelp_bin).to(device)

In [24]:
bert_checkpoint_finetuned_yelp_full = '/home/jasko/resilient_nlp/output/bert-base-uncased-yelp_full/checkpoint-1500'
if 'bert' in model_types and 'yelp_full' in tasks:
    model_finetuned['bert']['yelp_full'] = BertForSequenceClassification.from_pretrained(bert_checkpoint_finetuned_yelp_full).to(device)

In [25]:
roberta_checkpoint = "roberta-base"
tokenizer['roberta'] = AutoTokenizer.from_pretrained(roberta_checkpoint)
model_base['roberta'] = RobertaModel.from_pretrained(roberta_checkpoint).to(device)

In [26]:
roberta_checkpoint_finetuned_imdb = "jjezabek/roberta-base-imdb"
if 'roberta' in model_types and 'imdb' in tasks:
    model_finetuned['roberta']['imdb'] = RobertaForSequenceClassification.from_pretrained(roberta_checkpoint_finetuned_imdb).to(device)

In [27]:
roberta_checkpoint_finetuned_sst = '/home/jasko/resilient_nlp/output/roberta-base-sst/checkpoint-900'
if 'roberta' in model_types and 'sst' in tasks:
    model_finetuned['roberta']['sst'] = RobertaForSequenceClassification.from_pretrained(roberta_checkpoint_finetuned_sst).to(device)

In [28]:
roberta_checkpoint_finetuned_sst_bin = '/home/jasko/resilient_nlp/output/roberta-base-sst_bin/checkpoint-700'
if 'roberta' in model_types and 'sst_bin' in tasks:
    model_finetuned['roberta']['sst_bin'] = RobertaForSequenceClassification.from_pretrained(roberta_checkpoint_finetuned_sst_bin).to(device)

In [29]:
roberta_checkpoint_finetuned_yelp_bin = '/home/jasko/resilient_nlp/output/roberta-base-yelp_bin/checkpoint-2200'
if 'roberta' in model_types and 'yelp_bin' in tasks:
    model_finetuned['roberta']['yelp_bin'] = RobertaForSequenceClassification.from_pretrained(roberta_checkpoint_finetuned_yelp_bin).to(device)

In [30]:
roberta_checkpoint_finetuned_yelp_full = '/home/jasko/resilient_nlp/output/roberta-base-yelp_full/checkpoint-7500'
if 'roberta' in model_types and 'yelp_full' in tasks:
    model_finetuned['roberta']['yelp_full'] = RobertaForSequenceClassification.from_pretrained(roberta_checkpoint_finetuned_yelp_full).to(device)

### RobEn clusterings (as baseline)

The first clustering is ConnComp (which very aggressively merges clusters). The second is AggClust, which uses a cost function to better preserve fidelity. The second one should generally be better.

In [31]:
roben_clustering = Clustering.from_pickle("../vocab100000_ed1.pkl")
roben_recoverer = ClusterRecovererWithPassthrough("cache", roben_clustering)
roben_clustering2 = Clustering.from_pickle("../vocab100000_ed1_gamma0.3.pkl")
roben_recoverer2 = ClusterRecovererWithPassthrough("cache", roben_clustering2)

## Model Prediction Helpers

In [32]:
max_sequence_length = 128
batch_size = 32

These are wrappers for standard (possibly finetuned) Huggingface models, using their normal tokenizers.

In [33]:
def standard_model_predict(tokenizer, model, sentences, recoverer, return_pred_tensor, recoverer_tokenize):
    if recoverer is not None:
        if recoverer_tokenize:
            tok = nltk.tokenize.treebank.TreebankWordTokenizer()
            sentences = [ " ".join(tok_list) for tok_list in tok.tokenize_sents(sentences) ]
        sentences = [ recoverer.recover(s.lower()) for s in sentences ]
        if recoverer_tokenize:
            detok = nltk.tokenize.treebank.TreebankWordDetokenizer()
            sentences = [ detok.detokenize(s.split(" ")) for s in sentences]
    tokenized = tokenizer(sentences, truncation=True, padding='max_length', max_length=max_sequence_length,
                          return_tensors='pt')
    tokenized = { k: v.to(device) for k, v in tokenized.items() }
    preds = model(**tokenized)
    if return_pred_tensor:
        return preds
    else:
        return torch.argmax(preds.logits, dim=1)

def wrap_standard_model(tokenizer, model, recoverer=None, return_pred_tensor=True, recoverer_tokenize=False):
    return lambda sentences: standard_model_predict(tokenizer, model, sentences, recoverer, return_pred_tensor,
                                                    recoverer_tokenize)

This is a wrapper for the machine trained tokenizer+embedder (aka MockingBERT)

In [34]:
def mltokenizer_model_predict(runner, model, cls_embedding, sep_embedding, pad_embedding, sentences, return_pred_tensor):
    # Truncate and lower case. Truncation is for performance only
    sentences = [ s.lower()[:8*max_sequence_length] for s in sentences]
    embedding = runner.embed(sentences=sentences,
        start_token=cls_embedding, end_token=sep_embedding, pad_token=pad_embedding,
        max_tokens=max_sequence_length)
    preds = model(inputs_embeds=embedding['inputs_embeds'], attention_mask=embedding['attention_mask'])
    if return_pred_tensor:
        return preds
    else:
        return torch.argmax(preds.logits, dim=1)

def wrap_mltokenizer_model(mltokenizer_prefix, tokenizer, model, cf_embedding, type, return_pred_tensor=True):
    filename = "../{}.pth".format(mltokenizer_prefix)
    runner = ExperimentRunner(device, model_filename=filename)
    if type == 'bert':
        cls_token_id = tokenizer.vocab['[CLS]']
        sep_token_id = tokenizer.vocab['[SEP]']
        pad_token_id = tokenizer.vocab['[PAD]']
    elif type == 'roberta':
        cls_token_id = tokenizer.vocab['<s>']
        sep_token_id = tokenizer.vocab['</s>']
        pad_token_id = tokenizer.vocab['<pad>']
    cls_embedding = cf_embedding(torch.tensor([cls_token_id], device=device)).view(-1)
    sep_embedding = cf_embedding(torch.tensor([sep_token_id], device=device)).view(-1)
    pad_embedding = cf_embedding(torch.tensor([pad_token_id], device=device)).view(-1)
    
    return lambda sentences: mltokenizer_model_predict(runner, model, cls_embedding, sep_embedding,
                                                      pad_embedding, sentences, return_pred_tensor)

## Evaluation Helpers

Evaluates a wrapped model on a test set

In [35]:
@torch.no_grad()
def evaluate_model(model, test_set):
    num_batches = math.ceil(len(test_set) / batch_size)
    
    sentences = [ x['text'] for x in test_set ]
    labels = [ x['label'] for x in test_set ]
    pred_batches = []
    
    for i in tqdm(range(num_batches)):
        bs = i * batch_size
        be = bs + batch_size
        
        output = model(sentences[bs:be])
        
        pred_batches.append(torch.argmax(output.logits, dim=1).detach().cpu())
    preds = torch.cat(pred_batches)
    
    print(classification_report(labels, preds, digits=4))
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    
    return accuracy, f1

Evaluates a wrapped model on a stochastic, pseudo-adversarial test set. This means that each input sentence is replicated x times (typically 10) with randomized perturbations, and an attack is considered successful if *any* of the predictions is incorrect.

In [36]:
@torch.no_grad()
def evaluate_model_adv(model, test_sets):
    labels = [ x['label'] for x in test_sets[0] ]
    adv_preds = copy.copy(labels)
    accuracy_list = []
    f1_list = []
    
    for idx in tqdm(range(len(test_sets))):
        test_set = test_sets[idx]
        num_batches = math.ceil(len(test_set) / batch_size)
    
        sentences = [ x['text'] for x in test_set ]
        pred_batches = []
    
        for i in range(num_batches):
            bs = i * batch_size
            be = bs + batch_size
        
            output = model(sentences[bs:be])
        
            pred_batches.append(torch.argmax(output.logits, dim=1).detach().cpu())
        preds = torch.cat(pred_batches)
        
        for i in range(len(adv_preds)):
            if labels[i] != preds[i]:
                adv_preds[i] = preds[i]

        accuracy_list.append(accuracy_score(labels, adv_preds))
        f1_list.append(f1_score(labels, adv_preds, average='macro'))
    
    print(classification_report(labels, adv_preds, digits=4))    
    
    return accuracy_list, f1_list

Evaluates a model using WordScoreAttack

In [37]:
@torch.no_grad()
def evaluate_model_word_score(model, test_set, allow_whitespace_pert=True, report_prefix=None, word_scores_file=None):
    attacker = BertWordScoreAttack(
        WordScramblerPerturber(perturb_prob=1, weight_add=1, weight_drop=1, weight_swap=1,
                               weight_split_word=int(allow_whitespace_pert),
                               weight_merge_words=0),
        word_scores_file, model, tokenizer=None, max_sequence_length=max_sequence_length,
        attack_whitespace=allow_whitespace_pert,
    )

    res = attacker.attack(test_set, max_tokens_to_perturb=10, max_tries_per_token=4, mode=0, print_summary=False)

    if report_prefix is not None:
        res.to_csv(f"{report_prefix}_df.csv")
        with open(f"{report_prefix}_stats.json", "w") as f:
            json.dump(attacker.compute_attack_stats(), fp=f)            
    
    print(classification_report(res['ground_truth'], res['perturbed_preds'], digits=4))    
    
    accuracy = accuracy_score(res['ground_truth'], res['perturbed_preds'])
    f1 = f1_score(res['ground_truth'], res['perturbed_preds'], average='macro')
    
    return accuracy, f1

In [38]:
all_models = {
    'baseline': lambda task, type: wrap_standard_model(tokenizer[type], model_finetuned[type][task]),
    'baseline_all_pert': lambda task, type: wrap_standard_model(tokenizer[type], model_finetuned_all_pert[type][task]),
    'roben_1': lambda task, type: wrap_standard_model(tokenizer[type], model_finetuned[type][task], roben_recoverer),
    'roben_2': lambda task, type: wrap_standard_model(tokenizer[type], model_finetuned[type][task], roben_recoverer2),
    'roben_1_tok': lambda task, type: wrap_standard_model(tokenizer[type], model_finetuned[type][task], roben_recoverer, recoverer_tokenize=True),
    'roben_2_tok': lambda task, type: wrap_standard_model(tokenizer[type], model_finetuned[type][task], roben_recoverer2, recoverer_tokenize=True),
}

mltok_model_names = [
    '64k_lstm_clean_vanilla',
    '64k_lstm_no_whitespace_pert_vanilla',
    '64k_lstm_all_pert_vanilla',
    '64k_lstm_clean_finetuned',
    '64k_lstm_no_whitespace_pert_finetuned',
    '64k_lstm_all_pert_finetuned',
    '64k_cnn_no_whitespace_pert_finetuned',
    '2m_lstm_all_pert_finetuned',
    '32k_lstm_all_pert_finetuned_100ep',
]

for name in mltok_model_names:
    if name.endswith('_vanilla'):
        cf_embedding = lambda task, type: model_base[type].embeddings.word_embeddings
        filename = lambda task, type, name: f'output/{type}_{name}'
    else:
        cf_embedding = lambda task, type: model_finetuned[type][task].base_model.embeddings.word_embeddings
        filename = lambda task, type, name: f'output/{type}_{name}_{task}'
    # name=name is a hack to avoid Python late binding
    all_models[name] = lambda task, type, name=name, filename=filename, cf_embedding=cf_embedding: wrap_mltokenizer_model(filename(task, type, name), tokenizer[type], model_finetuned[type][task], cf_embedding(task, type), type)

In [39]:
evaluations = [
    'clean',
    'stochastic_no_ws',
    'stochastic_incl_ws',
    'word_score_no_ws',
    'word_score_incl_ws',
]

In [40]:
model_task_ids = [ f"{model}_{type}_{task}" for task, type, model in itertools.product(tasks, model_types, all_models.keys()) ]

accuracy_df = pd.DataFrame(columns=evaluations, index=model_task_ids)
f1_df = pd.DataFrame(columns=evaluations, index=model_task_ids)

for task, type in itertools.product(tasks, model_types):
    for cur_model_name, cur_model_factory in all_models.items():
        try:
            cur_model = cur_model_factory(task, type)
        except:
            print(f'Failed loading model {cur_model_name} on {type} for task {task}, skipping')
            accuracy_df.drop(f"{cur_model_name}_{type}_{task}", inplace=True)
            f1_df.drop(f"{cur_model_name}_{type}_{task}", inplace=True)
            continue
        for cur_evaluation in evaluations:
            print(f'Evaluating model {cur_model_name} on {type} on {cur_evaluation} for task {task}')
            start_time = time.time()
            random.seed(11)
            if cur_evaluation == 'clean':
                acc, f1 = evaluate_model(cur_model, sampled_test_set[task])
            elif cur_evaluation.startswith('stochastic_'):
                if cur_evaluation == 'stochastic_no_ws':
                    acc_list, f1_list = evaluate_model_adv(cur_model, sampled_test_set_adv_no_ws[task])
                elif cur_evaluation == 'stochastic_incl_ws':
                    acc_list, f1_list = evaluate_model_adv(cur_model, sampled_test_set_adv_incl_ws[task])
                acc = acc_list[-1]
                f1 = f1_list[-1]
                with open(f"../output/eval/{cur_model_name}_{type}_{task}_{cur_evaluation}_acc_list.json", "w") as f:
                    json.dump(acc_list, fp=f)
                with open(f"../output/eval/{cur_model_name}_{type}_{task}_{cur_evaluation}_f1_list.json", "w") as f:
                    json.dump(f1_list, fp=f)
            elif cur_evaluation.startswith('word_score_'):
                if cur_evaluation == 'word_score_no_ws':
                    acc, f1 = evaluate_model_word_score(cur_model, sampled_test_set[task], allow_whitespace_pert=False,
                                                        report_prefix=f"../output/eval/{cur_model_name}_{type}_{task}_{cur_evaluation}",
                                                        word_scores_file=f"../output/{task}_word_scores.json")
                elif cur_evaluation == 'word_score_incl_ws':
                    acc, f1 = evaluate_model_word_score(cur_model, sampled_test_set[task], allow_whitespace_pert=True,
                                                        report_prefix=f"../output/eval/{cur_model_name}_{type}_{task}_{cur_evaluation}",
                                                        word_scores_file=f"../output/{task}_word_scores.json")

            accuracy_df[cur_evaluation][f"{cur_model_name}_{type}_{task}"] = acc
            f1_df[cur_evaluation][f"{cur_model_name}_{type}_{task}"] = f1
            end_time = time.time()
            print(f"Evaluation took {end_time-start_time} seconds")
        del cur_model

Evaluating model baseline on bert on clean for task yelp_bin


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  5.04it/s]


              precision    recall  f1-score   support

           0     0.9346    0.9720    0.9529       250
           1     0.9708    0.9320    0.9510       250

    accuracy                         0.9520       500
   macro avg     0.9527    0.9520    0.9520       500
weighted avg     0.9527    0.9520    0.9520       500

Evaluation took 3.320714235305786 seconds
Evaluating model baseline on bert on stochastic_no_ws for task yelp_bin


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:25<00:00,  2.59s/it]


              precision    recall  f1-score   support

           0     0.8626    0.9040    0.8828       250
           1     0.8992    0.8560    0.8770       250

    accuracy                         0.8800       500
   macro avg     0.8809    0.8800    0.8799       500
weighted avg     0.8809    0.8800    0.8799       500

Evaluation took 26.032347917556763 seconds
Evaluating model baseline on bert on stochastic_incl_ws for task yelp_bin


...

              precision    recall  f1-score   support

           0     0.9258    0.9480    0.9368       250
           1     0.9467    0.9240    0.9352       250

    accuracy                         0.9360       500
   macro avg     0.9363    0.9360    0.9360       500
weighted avg     0.9363    0.9360    0.9360       500

Evaluation took 220.16953778266907 seconds
Evaluating model 64k_lstm_all_pert_finetuned on roberta on word_score_no_ws for task yelp_bin


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [13:58<00:00,  1.68s/it]


              precision    recall  f1-score   support

           0     0.8425    0.9200    0.8795       250
           1     0.9119    0.8280    0.8679       250

    accuracy                         0.8740       500
   macro avg     0.8772    0.8740    0.8737       500
weighted avg     0.8772    0.8740    0.8737       500

Evaluation took 838.2970867156982 seconds
Evaluating model 64k_lstm_all_pert_finetuned on roberta on word_score_incl_ws for task yelp_bin


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [14:13<00:00,  1.71s/it]

              precision    recall  f1-score   support

           0     0.8498    0.9280    0.8872       250
           1     0.9207    0.8360    0.8763       250

    accuracy                         0.8820       500
   macro avg     0.8853    0.8820    0.8817       500
weighted avg     0.8853    0.8820    0.8817       500

Evaluation took 854.0374884605408 seconds
Failed loading model 64k_cnn_no_whitespace_pert_finetuned on roberta for task yelp_bin, skipping
Failed loading model 2m_lstm_all_pert_finetuned on roberta for task yelp_bin, skipping
Failed loading model 32k_lstm_all_pert_finetuned_100ep on roberta for task yelp_bin, skipping


In [41]:
accuracy_df

,clean,stochastic_no_ws,stochastic_incl_ws,word_score_no_ws,word_score_incl_ws
baseline_bert_yelp_bin,0.952,0.88,0.874,0.712,0.704
roben_1_bert_yelp_bin,0.884,0.818,0.768,0.78,0.662
roben_2_bert_yelp_bin,0.92,0.878,0.838,0.838,0.706
roben_1_tok_bert_yelp_bin,0.866,0.804,0.75,0.806,0.648
roben_2_tok_bert_yelp_bin,0.904,0.872,0.812,0.864,0.71
64k_lstm_clean_vanilla_bert_yelp_bin,0.95,0.898,0.88,0.752,0.716
64k_lstm_no_whitespace_pert_vanilla_bert_yelp_bin,0.958,0.932,0.904,0.892,0.782
64k_lstm_all_pert_vanilla_bert_yelp_bin,0.952,0.932,0.936,0.886,0.898
64k_lstm_all_pert_finetuned_bert_yelp_bin,0.954,0.94,0.938,0.884,0.89
baseline_roberta_yelp_bin,0.964,0.896,0.9,0.76,0.78


In [42]:
f1_df

,clean,stochastic_no_ws,stochastic_incl_ws,word_score_no_ws,word_score_incl_ws
baseline_bert_yelp_bin,0.951981,0.879931,0.873987,0.711834,0.703697
roben_1_bert_yelp_bin,0.883635,0.817104,0.767371,0.778854,0.658722
roben_2_bert_yelp_bin,0.919896,0.877824,0.837453,0.837594,0.704866
roben_1_tok_bert_yelp_bin,0.865548,0.802979,0.748769,0.804416,0.643199
roben_2_tok_bert_yelp_bin,0.903778,0.871597,0.81102,0.863509,0.708403
64k_lstm_clean_vanilla_bert_yelp_bin,0.949995,0.898,0.879998,0.751678,0.715959
64k_lstm_no_whitespace_pert_vanilla_bert_yelp_bin,0.957998,0.931999,0.903975,0.891993,0.781992
64k_lstm_all_pert_vanilla_bert_yelp_bin,0.951993,0.931996,0.935996,0.885923,0.897951
64k_lstm_all_pert_finetuned_bert_yelp_bin,0.953998,0.939996,0.937994,0.883881,0.889947
baseline_roberta_yelp_bin,0.964,0.895998,0.899986,0.759615,0.779647


In [43]:
accuracy_df.to_csv("../output/grid_accuracy.csv")

In [44]:
f1_df.to_csv("../output/grid_f1.csv")

-----